In [1]:
!pip install ogb
!pip install memory-profiler

In [2]:
!pip uninstall -y torch torchvision torchaudio torch-geometric torch-sparse torch-scatter torch-cluster torch-spline-conv
!pip uninstall -y torch torchvision torchaudio torch-geometric

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Successfully uninstalled torch-2.0.1+cu118
Found existing installation: torchvision 0.15.2+cu118
Uninstalling torchvision-0.15.2+cu118:
  Successfully uninstalled torchvision-0.15.2+cu118
Found existing installation: torchaudio 2.0.2+cu118
Uninstalling torchaudio-2.0.2+cu118:
  Successfully uninstalled torchaudio-2.0.2+cu118
Found existing installation: torch-geometric 2.6.1
Uninstalling torch-geometric-2.6.1:
  Successfully uninstalled torch-geometric-2.6.1
Found existing installation: torch-sparse 0.6.18
Uninstalling torch-sparse-0.6.18:
  Successfully uninstalled torch-sparse-0.6.18
Found existing installation: torch-scatter 2.1.2+pt20cu118
Uninstalling torch-scatter-2.1.2+pt20cu118:
  Successfully uninstalled torch-scatter-2.1.2+pt20cu118
Found existing installation: torch-cluster 1.6.3
Uninstalling torch-cluster-1.6.3:
  Successfully uninstalled torch-cluster-1.6.3
Found existing installation: torch-s

In [3]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-linux_x86_64.whl (2267.3 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.15.2%2Bcu118-cp310-cp310-linux_x86_64.whl (6.1 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.0.2%2Bcu118-cp310-cp310-linux_x86_64.whl (4.4 MB)


In [4]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-geometric


Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
  Using cached https://data.pyg.org/whl/torch-2.0.0%2Bcu118/torch_scatter-2.1.2%2Bpt20cu118-cp310-cp310-linux_x86_64.whl (10.2 MB)
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
  Using cached https://data.pyg.org/whl/torch-2.0.0%2Bcu118/torch_sparse-0.6.18%2Bpt20cu118-cp310-cp310-linux_x86_64.whl (4.9 MB)
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
  Using cached https://data.pyg.org/whl/torch-2.0.0%2Bcu118/torch_cluster-1.6.3%2Bpt20cu118-cp310-cp310-linux_x86_64.whl (3.3 MB)
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
  Using cached https://data.pyg.org/whl/torch-2.0.0%2Bcu118/torch_spline_conv-1.2.2%2Bpt20cu118-cp310-cp310-linux_x86_64.whl (886 kB)
  Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


In [5]:
!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.0.1+cu118.html

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html


In [6]:
!pip uninstall -y dgl
!pip install dgl-cu117 -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html


In [7]:
!apt-get install libcusparse11

E: Nie udało się otworzyć pliku blokady /var/lib/dpkg/lock-frontend - open (13: Brak dostępu)
E: Nie udało się pozyskać blokady interfejsu dpkg (/var/lib/dpkg/lock-frontend), czy użyto uprawnień administratora?


In [8]:
import dgl
import torch

print("DGL version:", dgl.__version__)
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Czy GPU jest dostępne:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Używana karta GPU:", torch.cuda.get_device_name(0))

DGL version: 0.9.1post1
PyTorch version: 2.0.1+cu118
CUDA available: True
Czy GPU jest dostępne: True
Używana karta GPU: NVIDIA GeForce RTX 4070 Ti


In [9]:
# Importowanie bibliotek
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
from torch_geometric.transforms import NormalizeFeatures
from ogb.nodeproppred import PygNodePropPredDataset, NodePropPredDataset
from torch_sparse import SparseTensor
from torch_geometric.utils import to_undirected
import torch_geometric.transforms as T
from torch_sparse import SparseTensor
import pandas as pd
import csv
import os
import time
import memory_profiler

/home/macierz/s185569/eniu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
def load_data(library, matrix_format, dataset_name="ogbn-arxiv"):
    if dataset_name.startswith("ogbn-"):  
        dataset = NodePropPredDataset(name=dataset_name)
        split_idx = dataset.get_idx_split()  # Podziały na train/valid/test
        graph, labels = dataset[0]  # Pobieranie: graf i etykiety

        if library == "PyG":
            import torch_geometric.data as pyg_data
            edge_index = torch.tensor(graph["edge_index"], dtype=torch.long)
            x = torch.tensor(graph["node_feat"], dtype=torch.float)
            y = torch.tensor(labels, dtype=torch.long)

            data = pyg_data.Data(x=x, edge_index=to_undirected(edge_index), y=y)
            data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
            data.val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
            data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
            data.train_mask[split_idx["train"]] = True
            data.val_mask[split_idx["valid"]] = True
            data.test_mask[split_idx["test"]] = True

            # Konwersja do wybranego formatu
            data = convert_pyg_format(data, matrix_format)
            return data, dataset_name, dataset

        elif library == "DGL":
            graph_dgl = dgl.graph((graph["edge_index"][0], graph["edge_index"][1]))
            graph_dgl.ndata['feat'] = torch.tensor(graph["node_feat"], dtype=torch.float)
            graph_dgl.ndata['label'] = torch.tensor(labels, dtype=torch.long)
            graph_dgl.ndata['train_mask'] = torch.zeros(graph_dgl.num_nodes(), dtype=torch.bool)
            graph_dgl.ndata['val_mask'] = torch.zeros(graph_dgl.num_nodes(), dtype=torch.bool)
            graph_dgl.ndata['test_mask'] = torch.zeros(graph_dgl.num_nodes(), dtype=torch.bool)
            graph_dgl.ndata['train_mask'][split_idx["train"]] = True
            graph_dgl.ndata['val_mask'][split_idx["valid"]] = True
            graph_dgl.ndata['test_mask'][split_idx["test"]] = True

            # Konwersja do wybranego formatu
            graph_dgl = convert_dgl_format(graph_dgl, matrix_format)
            return graph_dgl, dataset_name, dataset

    else:
        # Obsługa innych zbiorów danych
        return 1,2,3

In [67]:
# from torch_sparse import SparseTensor

# def convert_pyg_format(data, matrix_format):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#     if matrix_format == "COO":
#         print("Data is in COO format.")
#         print("Sample edge_index (COO):")
#         print("Rows:", data.edge_index[0][:10])
#         print("Cols:", data.edge_index[1][:10])
#         return data

#     elif matrix_format == "CSR":
#         print("Converting to CSR format.")
#         adj_csr = SparseTensor.from_edge_index(
#             data.edge_index, sparse_sizes=(data.num_nodes, data.num_nodes)
#         ).to(device)
#         data.adj_t = adj_csr
#         print("Sample CSR format:")
#         print("Indptr:", adj_csr.storage.rowptr()[:10])
#         print("Cols:", adj_csr.storage.col()[:10])
#         print("Values:", adj_csr.storage.value()[:10])
#         print(f"Type of data.adj_t after conversion: {type(data.adj_t)}")
#         return data

#     elif matrix_format == "CSC":
#         print("Converting to CSC format.")
#         adj_csc = SparseTensor.from_edge_index(
#             data.edge_index, sparse_sizes=(data.num_nodes, data.num_nodes)
#         ).to(device).t()  # Transpozycja do formatu CSC
#         data.adj_t = adj_csc
#         print("Sample CSC format:")
#         print("Indptr:", adj_csc.storage.rowptr()[:10])
#         print("Rows:", adj_csc.storage.col()[:10])
#         print("Values:", adj_csc.storage.value()[:10])
#         print(f"Type of data.adj_t after conversion: {type(data.adj_t)}")
#         return data

#     else:
#         raise ValueError(f"Unsupported matrix format: {matrix_format}")


In [76]:
from torch_sparse import SparseTensor

def convert_pyg_format(data, matrix_format):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device = "cpu"
    if matrix_format == "COO":
        print("Data is in COO format.")
        print("Sample edge_index (COO):")
        print("Rows:", data.edge_index[0][:10])
        print("Cols:", data.edge_index[1][:10])
        return data

    elif matrix_format == "CSR":
        print("Converting to CSR format.")
        adj_csr = SparseTensor.from_edge_index(
            data.edge_index,
            sparse_sizes=(data.num_nodes, data.num_nodes)
        ).to(device)
        data.adj_t = adj_csr

        print("Sample CSR format:")
        print("Indptr:", adj_csr.storage.rowptr()[:10])
        print("Cols:", adj_csr.storage.col()[:10])
        print(f"Type of data.adj_t after conversion: {type(data.adj_t)}")
        return data

    elif matrix_format == "CSC":
        print("Converting to CSC format.")
        adj_csc = SparseTensor.from_edge_index(
            data.edge_index,
            sparse_sizes=(data.num_nodes, data.num_nodes)
        ).to(device).t()  # Transpose to get CSC
        data.adj_t = adj_csc

        print("Sample CSC format:")
        print("Indptr:", adj_csc.storage.rowptr()[:10])
        print("Cols:", adj_csc.storage.col()[:10])
        print(f"Type of data.adj_t after conversion: {type(data.adj_t)}")
        return data

    else:
        raise ValueError(f"Unsupported matrix format: {matrix_format}")


In [77]:
# from torch_sparse import SparseTensor

# def convert_pyg_format(data, matrix_format):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#     if matrix_format == "COO":
#         print("Data is in COO format.")
#         return data

#     elif matrix_format == "CSR":
#         print("Converting to CSR format.")
#         adj_csr = SparseTensor.from_edge_index(
#             data.edge_index, sparse_sizes=(data.num_nodes, data.num_nodes)
#         ).to(device).to_sparse_csr()
#         data.adj_t = adj_csr
#         print(f"Type of data.adj_t after conversion: {type(data.adj_t)}")
#         return data

#     elif matrix_format == "CSC":
#         print("Converting to CSC format.")
#         adj_csc = SparseTensor.from_edge_index(
#             data.edge_index, sparse_sizes=(data.num_nodes, data.num_nodes)
#         ).to(device).to_sparse_csc()
#         data.adj_t = adj_csc
#         print(f"Type of data.adj_t after conversion: {type(data.adj_t)}")
#         return data

#     else:
#         raise ValueError(f"Unsupported matrix format: {matrix_format}")


In [78]:
# from torch.sparse import coo_tensor, csr_tensor, csc_tensor
# def convert_pyg_format(data, matrix_format):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#     if matrix_format == "COO":
#         print("Data is in COO format.")
#         # Wyświetlenie próbki danych w formacie COO
#         print("Sample edge_index (COO):")
#         print("Rows:", data.edge_index[0][:10])
#         print("Cols:", data.edge_index[1][:10])
#         return data  # COO to domyślny format PyG

#     elif matrix_format == "CSR":
#         print("Converting to CSR format.")
#         row = data.edge_index[0].to(torch.int64)
#         col = data.edge_index[1].to(torch.int64)
#         values = torch.ones(row.size(0), device=device)
#         adj_csr = csr_tensor((values, row, col), size=(data.num_nodes, data.num_nodes), device=device)
#         data.adj_t = adj_csr

#         # Wyświetlenie próbki danych w formacie CSR
#         print("Sample CSR format:")
#         print("Indptr:", adj_csr.crow_indices()[:10])
#         print("Cols:", adj_csr.col_indices()[:10])
#         print("Values:", adj_csr.values()[:10])
#         return data

#     elif matrix_format == "CSC":
#         print("Converting to CSC format.")
#         row = data.edge_index[0].to(torch.int64)
#         col = data.edge_index[1].to(torch.int64)
#         values = torch.ones(row.size(0), device=device)
#         adj_csc = csc_tensor((values, row, col), size=(data.num_nodes, data.num_nodes), device=device)
#         data.adj_t = adj_csc

#         # Wyświetlenie próbki danych w formacie CSC
#         print("Sample CSC format:")
#         print("Indptr:", adj_csc.ccol_indices()[:10])
#         print("Rows:", adj_csc.row_indices()[:10])
#         print("Values:", adj_csc.values()[:10])
#         return data

#     else:
#         raise ValueError(f"Unsupported matrix format: {matrix_format}")


In [79]:
# from torch_sparse import SparseTensor

# def convert_pyg_format(data, matrix_format):
#     """
#     Converts the data object in PyTorch Geometric to the specified sparse matrix format.
    
#     Args:
#         data (torch_geometric.data.Data): The PyG data object.
#         matrix_format (str): The desired format ('COO', 'CSR', 'CSC').
        
#     Returns:
#         torch_geometric.data.Data: Updated PyG data object with converted format.
#     """
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#     if matrix_format == "COO":
#         print("Converting to COO format")
#         if hasattr(data, 'adj_t'):
#             del data.adj_t  # Remove any previous formats
#         return data

#     elif matrix_format == "CSR":
#         print("Converting to CSR format")
#         # Convert edge_index to SparseTensor
#         adj_csr = SparseTensor.from_edge_index(
#             row=data.edge_index[0],
#             col=data.edge_index[1],
#             sparse_sizes=(data.num_nodes, data.num_nodes)
#         ).to(device)
#         data.adj_t = adj_csr  # Store as adj_t in CSR format
#         return data

#     elif matrix_format == "CSC":
#         print("Converting to CSC format")
#         # Convert edge_index to SparseTensor
#         adj_csc = SparseTensor.from_edge_index(
#             row=data.edge_index[0],
#             col=data.edge_index[1],
#             sparse_sizes=(data.num_nodes, data.num_nodes)
#         ).to(device).t()  # Use transpose for CSC
#         data.adj_t = adj_csc  # Store as adj_t in CSC format
#         return data

#     else:
#         raise ValueError(f"Unsupported matrix format: {matrix_format}")


In [80]:
# #Funkcja obsługująca formaty macierzy dla PyTorch Geometric (PyG)
# def convert_pyg_format(data, matrix_format):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     #device = "cpu"
#     if matrix_format == "COO":
#         # Dane już w formacie COO (edge_index)
#         return data
#     elif matrix_format == "CSR":
#         # Konwersja do CSR (SparseTensor)
#         data.adj_t = SparseTensor(row=data.edge_index[0], col=data.edge_index[1], value=None).to(device)
#         return data
#     elif matrix_format == "CSC":
#         # Konwersja do CSR, a potem transpozycja do CSC
#         data.adj_t = SparseTensor(row=data.edge_index[0], col=data.edge_index[1], value=None).to(device)
#         data.adj_t = data.adj_t.t()  # Użycie t() zamiast transpose()
#         return data
#     else:
#         raise ValueError("Nieznany format macierzy dla PyG: {}".format(matrix_format))

# # # Funkcja obsługująca formaty macierzy dla DGL
# # def convert_dgl_format(graph, matrix_format):
# #     if matrix_format == "COO":
# #         # DGL domyślnie obsługuje COO, nie trzeba konwertować
# #         return graph
# #     elif matrix_format == "CSR":
# #         # Konwersja do formatu CSR
# #         return dgl.to_block(dgl.to_simple(graph))  # Konwersja na CSR
# #     elif matrix_format == "CSC":
# #         graph = dgl.add_self_loop(graph)  # DGL nie wspiera natywnie CSC, dodanie self-loop
# #         return graph
# #     else:
# #         raise ValueError("Nieznany format macierzy dla DGL: {}".format(matrix_format))


In [81]:
from torch_geometric.nn import GCNConv, SAGEConv, GATConv
import torch.nn.functional as F
from torch_geometric.loader import NeighborLoader
import torch
import csv
import os
import time

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

class GraphSAGE(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=8):
        super(GAT, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [82]:
import torch.optim as optim
import torch.nn.functional as F
import gc

In [83]:
def train_with_batching(model, data, num_epochs, batch_size=32):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    device = "cpu"
    model.to(device)
    model.train()

    train_loader = NeighborLoader(
        data,
        num_neighbors=[5, 5],
        batch_size=batch_size,
        input_nodes=data.train_mask,
        shuffle=True
    )
    
    total_time = 0
    for epoch in range(num_epochs):
        epoch_loss = 0
        start_time = time.time()

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            out = model(batch.x, batch.edge_index)
            out = out[batch.train_mask]

            # Upewnij się, że batch.y jest 1-wymiarowy
            target = batch.y[batch.train_mask]
            if target.dim() > 1:
                target = target.squeeze(dim=-1)

            loss = F.cross_entropy(out, target)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        scheduler.step()
        end_time = time.time()
        total_time += (end_time - start_time)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Time: {end_time - start_time:.2f}s")

    return total_time


In [84]:
def evaluate_model(model, data):
    model.eval()
    inference_time = 0
    correct = 0
    total = 0

    test_loader = NeighborLoader(
        data,
        num_neighbors=[5, 5],
        batch_size=32,
        input_nodes=data.test_mask,
        shuffle=False
    )
    device = "cuda" if torch.cuda.is_available() else "cpu"
    device = "cpu"
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            start_time = time.time()
            out = model(batch.x, batch.edge_index)
            out = out[batch.test_mask]
            end_time = time.time()
            inference_time += (end_time - start_time)

            pred = out.argmax(dim=1)
            correct += (pred == batch.y[batch.test_mask]).sum().item()
            total += batch.test_mask.sum().item()

    accuracy = correct / total
    return accuracy, inference_time

In [85]:
def run_experiments():
    models = {
        'GCN': GCN,
        'GraphSAGE': GraphSAGE,
        'GAT': GAT
    }
    formats = ["COO", "CSR", "CSC"]
   # models = {'GAT': GAT}
    libraries = ["PyG"]

    results = []
    file_exists = os.path.isfile('experiment_results_updated_h.csv')
    device = "cuda" if torch.cuda.is_available() else "cpu"
    device = "cpu"

    with open('experiment_results_updated_h.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Model", "Library", "Format", "Dataset", "E2E Time (s)", "Training Time (s)", "Inference Time (s)", "Validation Accuracy", "Device"])

        for model_name, model_class in models.items():
            for lib in libraries:
                for fmt in formats:
                    start_e2e_time = time.time()
                    data, dataset_name, dataset = load_data(lib, fmt)
                    print(f"Running experiment for {model_name} with {lib} using {fmt} format on {dataset_name}.")

                    model = model_class(
                        input_dim=data.num_node_features,
                        hidden_dim=1024,
                        output_dim=dataset.num_classes
                    )

                    train_time = train_with_batching(model, data, num_epochs=3, batch_size=32)
                    accuracy, inference_time = evaluate_model(model, data)

                    end_e2e_time = time.time()
                    e2e_time = end_e2e_time - start_e2e_time

                    writer.writerow([model_name, lib, fmt, dataset_name, e2e_time, train_time, inference_time, accuracy, device])
                    results.append([model_name, lib, fmt, dataset_name, e2e_time, train_time, inference_time, accuracy, device])

                    print(f"Finished {model_name} with {lib} using {fmt} format on {dataset_name}.")

    return results

In [86]:
results = run_experiments()

Data is in COO format.
Sample edge_index (COO):
Rows: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Cols: tensor([ 411,  640, 1162, 1897, 3396, 3787, 4692, 5037, 5537, 5611])
Running experiment for GCN with PyG using COO format on ogbn-arxiv.
Epoch 1/3, Loss: 4106.3487, Time: 141.00s
Epoch 2/3, Loss: 3848.9239, Time: 288.95s
Epoch 3/3, Loss: 3822.2105, Time: 287.05s
Finished GCN with PyG using COO format on ogbn-arxiv.
Converting to CSR format.
Sample CSR format:
Indptr: tensor([  0, 291, 293, 307, 309, 315, 316, 324, 326, 327])
Cols: tensor([ 411,  640, 1162, 1897, 3396, 3787, 4692, 5037, 5537, 5611])
Type of data.adj_t after conversion: <class 'torch_sparse.tensor.SparseTensor'>
Running experiment for GCN with PyG using CSR format on ogbn-arxiv.
Epoch 1/3, Loss: 4093.5759, Time: 133.60s
Epoch 2/3, Loss: 3856.0517, Time: 282.19s
Epoch 3/3, Loss: 3809.1559, Time: 285.52s
Finished GCN with PyG using CSR format on ogbn-arxiv.
Converting to CSC format.
Sample CSC format:
Indptr: tensor([  0, 291

In [72]:
results = run_experiments()

Data is in COO format.
Sample edge_index (COO):
Rows: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Cols: tensor([ 411,  640, 1162, 1897, 3396, 3787, 4692, 5037, 5537, 5611])
Running experiment for GCN with PyG using COO format on ogbn-arxiv.
Epoch 1/3, Loss: 4105.4658, Time: 5.41s
Epoch 2/3, Loss: 3853.4831, Time: 6.17s
Epoch 3/3, Loss: 3814.8776, Time: 6.04s
Finished GCN with PyG using COO format on ogbn-arxiv.
Converting to CSR format.
Sample CSR format:
Indptr: tensor([  0, 291, 293, 307, 309, 315, 316, 324, 326, 327], device='cuda:0')
Cols: tensor([ 411,  640, 1162, 1897, 3396, 3787, 4692, 5037, 5537, 5611],
       device='cuda:0')
Type of data.adj_t after conversion: <class 'torch_sparse.tensor.SparseTensor'>
Running experiment for GCN with PyG using CSR format on ogbn-arxiv.
Epoch 1/3, Loss: 4105.9567, Time: 6.64s
Epoch 2/3, Loss: 3849.8000, Time: 6.67s
Epoch 3/3, Loss: 3810.9957, Time: 6.14s
Finished GCN with PyG using CSR format on ogbn-arxiv.
Converting to CSC format.
Sample CSC for

In [18]:
#results = run_experiments()

In [75]:
#results = run_experiments()

Running experiment for GCN with PyG using COO format on ogbn-arxiv.
Epoch 1/3, Loss: 4106.9529, Time: 6.43s
Epoch 2/3, Loss: 3852.1607, Time: 6.39s
Epoch 3/3, Loss: 3816.9834, Time: 6.39s
Finished GCN with PyG using COO format on ogbn-arxiv.
Running experiment for GCN with PyG using CSR format on ogbn-arxiv.
Epoch 1/3, Loss: 4105.9370, Time: 7.01s
Epoch 2/3, Loss: 3852.6665, Time: 6.27s
Epoch 3/3, Loss: 3819.2325, Time: 6.15s
Finished GCN with PyG using CSR format on ogbn-arxiv.
Running experiment for GCN with PyG using CSC format on ogbn-arxiv.
Epoch 1/3, Loss: 4110.3234, Time: 6.36s
Epoch 2/3, Loss: 3857.2722, Time: 6.52s
Epoch 3/3, Loss: 3822.0606, Time: 6.42s
Finished GCN with PyG using CSC format on ogbn-arxiv.
Running experiment for GraphSAGE with PyG using COO format on ogbn-arxiv.
Epoch 1/3, Loss: 4030.3633, Time: 5.25s
Epoch 2/3, Loss: 3802.1511, Time: 4.89s
Epoch 3/3, Loss: 3766.2104, Time: 5.62s
Finished GraphSAGE with PyG using COO format on ogbn-arxiv.
Running experiment f

In [32]:
# import torch

# def convert_sparse_format(tensor, from_format, to_format):
#     """
#     Converts a sparse tensor between different formats: COO, CSR, CSC.

#     Args:
#         tensor (torch.Tensor): Input sparse tensor.
#         from_format (str): Format of the input tensor ('COO', 'CSR', 'CSC').
#         to_format (str): Desired output format ('COO', 'CSR', 'CSC').

#     Returns:
#         torch.Tensor: Sparse tensor in the desired format.

#     Raises:
#         ValueError: If an unsupported format is specified.
#     """
#     if from_format == to_format:
#         return tensor

#     if from_format == "COO":
#         if to_format == "CSR":
#             return tensor.to_sparse_csr()
#         elif to_format == "CSC":
#             return tensor.to_sparse_csc()
#     elif from_format == "CSR":
#         if to_format == "COO":
#             return tensor.to_sparse()
#         elif to_format == "CSC":
#             return tensor.to_sparse().to_sparse_csc()
#     elif from_format == "CSC":
#         if to_format == "COO":
#             return tensor.to_sparse()
#         elif to_format == "CSR":
#             return tensor.to_sparse().to_sparse_csr()

#     raise ValueError(f"Unsupported conversion from {from_format} to {to_format}")

# # Test function
# def test_sparse_conversion():
#     # Create a manual COO sparse tensor
#     indices = torch.tensor([[0, 1, 1], [2, 0, 2]])  # Row and column indices
#     values = torch.tensor([3, 4, 5])  # Non-zero values
#     size = (3, 3)  # Shape of the matrix
    
#     coo_tensor = torch.sparse_coo_tensor(indices, values, size)

#     # Convert to CSR and CSC
#     csr_tensor = convert_sparse_format(coo_tensor, "COO", "CSR")
#     csc_tensor = convert_sparse_format(coo_tensor, "COO", "CSC")

#     # Back to COO
#     coo_from_csr = convert_sparse_format(csr_tensor, "CSR", "COO")
#     coo_from_csc = convert_sparse_format(csc_tensor, "CSC", "COO")

#     # Print results
#     print("Original COO tensor:")
#     print(coo_tensor)

#     print("Converted to CSR:")
#     print(csr_tensor)

#     print("Converted to CSC:")
#     print(csc_tensor)

#     print("Converted back to COO from CSR:")
#     print(coo_from_csr)

#     print("Converted back to COO from CSC:")
#     print(coo_from_csc)

# if __name__ == "__main__":
#     test_sparse_conversion()


Original COO tensor:
tensor(indices=tensor([[0, 1, 1],
                       [2, 0, 2]]),
       values=tensor([3, 4, 5]),
       size=(3, 3), nnz=3, layout=torch.sparse_coo)
Converted to CSR:
tensor(crow_indices=tensor([0, 1, 3, 3]),
       col_indices=tensor([2, 0, 2]),
       values=tensor([3, 4, 5]), size=(3, 3), nnz=3, layout=torch.sparse_csr)
Converted to CSC:
tensor(ccol_indices=tensor([0, 1, 1, 3]),
       row_indices=tensor([1, 0, 1]),
       values=tensor([4, 3, 5]), size=(3, 3), nnz=3, layout=torch.sparse_csc)
Converted back to COO from CSR:
tensor(indices=tensor([[0, 1, 1],
                       [2, 0, 2]]),
       values=tensor([3, 4, 5]),
       size=(3, 3), nnz=3, layout=torch.sparse_coo)
Converted back to COO from CSC:
tensor(indices=tensor([[1, 0, 1],
                       [0, 2, 2]]),
       values=tensor([4, 3, 5]),
       size=(3, 3), nnz=3, layout=torch.sparse_coo)


/tmp/ipykernel_68974/3697929816.py:23: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return tensor.to_sparse_csr()


In [36]:
from ogb.nodeproppred import NodePropPredDataset
import torch

# Function to convert sparse formats
def convert_sparse_format(tensor, from_format, to_format):
    """
    Converts a sparse tensor between different formats: COO, CSR, CSC.

    Args:
        tensor (torch.Tensor): Input sparse tensor.
        from_format (str): Format of the input tensor ('COO', 'CSR', 'CSC').
        to_format (str): Desired output format ('COO', 'CSR', 'CSC').

    Returns:
        torch.Tensor: Sparse tensor in the desired format.

    Raises:
        ValueError: If an unsupported format is specified.
    """
    if from_format == to_format:
        return tensor

    if from_format == "COO":
        if to_format == "CSR":
            return tensor.to_sparse_csr()
        elif to_format == "CSC":
            return tensor.to_sparse_csc()
    elif from_format == "CSR":
        if to_format == "COO":
            return tensor.to_sparse()
        elif to_format == "CSC":
            return tensor.to_sparse().to_sparse_csc()
    elif from_format == "CSC":
        if to_format == "COO":
            return tensor.to_sparse()
        elif to_format == "CSR":
            return tensor.to_sparse().to_sparse_csr()

    raise ValueError(f"Unsupported conversion from {from_format} to {to_format}")

# Function to load OGB dataset and convert sparse formats
def load_and_convert_ogb(dataset_name="ogbn-arxiv", to_format="CSR"):
    dataset = NodePropPredDataset(name=dataset_name)
    split_idx = dataset.get_idx_split()
    graph, labels = dataset[0]

    # Prepare COO tensor
    edge_index = torch.tensor(graph["edge_index"], dtype=torch.int64)
    values = torch.ones(edge_index.shape[1])
    size = (graph["num_nodes"], graph["num_nodes"])
    coo_tensor = torch.sparse_coo_tensor(edge_index, values, size)

    # Convert to the desired format
    converted_tensor = convert_sparse_format(coo_tensor, "COO", to_format)

    # Return split indices and converted tensor
    return split_idx, converted_tensor

# Test function
def test_sparse_conversion_with_ogb():
    dataset_name = "ogbn-arxiv"

    # Load and convert to CSR
    split_idx, csr_tensor = load_and_convert_ogb(dataset_name, "CSR")
    print("Converted to CSR format:")
    print("Indptr:", csr_tensor.crow_indices()[:10])
    print("Cols:", csr_tensor.col_indices()[:10])
    print("Values:", csr_tensor.values()[:10])

    # Convert back to COO
    coo_from_csr = convert_sparse_format(csr_tensor, "CSR", "COO")
    print("Converted back to COO from CSR:")
    print("Indices:", coo_from_csr.indices()[:, :10])
    print("Values:", coo_from_csr.values()[:10])

if __name__ == "__main__":
    test_sparse_conversion_with_ogb()


Converted to CSR format:
Indptr: tensor([ 0,  2,  3, 12, 12, 13, 13, 13, 14, 15])
Cols: tensor([ 52893,  93487, 141692,  18735,  22689,  50646,  80266, 119218, 134600,
        158293])
Values: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Converted back to COO from CSR:
Indices: tensor([[     0,      0,      1,      2,      2,      2,      2,      2,      2,
              2],
        [ 52893,  93487, 141692,  18735,  22689,  50646,  80266, 119218, 134600,
         158293]])
Values: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [38]:
from ogb.nodeproppred import NodePropPredDataset
import torch

# Function to convert sparse formats
def convert_sparse_format(tensor, from_format, to_format):
    """
    Converts a sparse tensor between different formats: COO, CSR, CSC.

    Args:
        tensor (torch.Tensor): Input sparse tensor.
        from_format (str): Format of the input tensor ('COO', 'CSR', 'CSC').
        to_format (str): Desired output format ('COO', 'CSR', 'CSC').

    Returns:
        torch.Tensor: Sparse tensor in the desired format.

    Raises:
        ValueError: If an unsupported format is specified.
    """
    if from_format == to_format:
        return tensor

    if from_format == "COO":
        if to_format == "CSR":
            return tensor.to_sparse_csr()
        elif to_format == "CSC":
            return tensor.to_sparse_csc()
    elif from_format == "CSR":
        if to_format == "COO":
            return tensor.to_sparse()
        elif to_format == "CSC":
            return tensor.to_sparse().to_sparse_csc()
    elif from_format == "CSC":
        if to_format == "COO":
            return tensor.to_sparse()
        elif to_format == "CSR":
            return tensor.to_sparse().to_sparse_csr()

    raise ValueError(f"Unsupported conversion from {from_format} to {to_format}")

# Function to load OGB dataset and convert sparse formats
def load_and_convert_ogb(dataset_name="ogbn-arxiv", to_format="CSR"):
    dataset = NodePropPredDataset(name=dataset_name)
    split_idx = dataset.get_idx_split()
    graph, labels = dataset[0]

    # Prepare COO tensor
    edge_index = torch.tensor(graph["edge_index"], dtype=torch.int64)
    values = torch.ones(edge_index.shape[1])
    size = (graph["num_nodes"], graph["num_nodes"])
    coo_tensor = torch.sparse_coo_tensor(edge_index, values, size)

    # Convert to the desired format
    converted_tensor = convert_sparse_format(coo_tensor, "COO", to_format)

    # Return split indices and converted tensor
    return split_idx, converted_tensor

# Test function
def test_sparse_conversion_with_ogb():
    dataset_name = "ogbn-arxiv"

    # Load and convert to CSR
    split_idx, csr_tensor = load_and_convert_ogb(dataset_name, "CSR")
    print("Converted to CSR format:")
    print("Indptr:", csr_tensor.crow_indices()[:10])
    print("Cols:", csr_tensor.col_indices()[:10])
    print("Values:", csr_tensor.values()[:10])

    # Convert back to COO
    coo_from_csr = convert_sparse_format(csr_tensor, "CSR", "COO")
    print("Converted back to COO from CSR:")
    print("Indices:", coo_from_csr.indices()[:, :10])
    print("Values:", coo_from_csr.values()[:10])

    # Load and convert to CSC
    split_idx, csc_tensor = load_and_convert_ogb(dataset_name, "CSC")
    print("Converted to CSC format:")
    print("Indptr:", csc_tensor.ccol_indices()[:10])
    print("Rows:", csc_tensor.row_indices()[:10])
    print("Values:", csc_tensor.values()[:10])

    # Convert back to COO
    coo_from_csc = convert_sparse_format(csc_tensor, "CSC", "COO")
    print("Converted back to COO from CSC:")
    print("Indices:", coo_from_csc.indices()[:, :10])
    print("Values:", coo_from_csc.values()[:10])

if __name__ == "__main__":
    test_sparse_conversion_with_ogb()


NameError: name 'dataset_name' is not defined

In [39]:
from ogb.nodeproppred import NodePropPredDataset
import torch

# Function to convert sparse formats
def convert_sparse_format(tensor, from_format, to_format):
    """
    Converts a sparse tensor between different formats: COO, CSR, CSC.

    Args:
        tensor (torch.Tensor): Input sparse tensor.
        from_format (str): Format of the input tensor ('COO', 'CSR', 'CSC').
        to_format (str): Desired output format ('COO', 'CSR', 'CSC').

    Returns:
        torch.Tensor: Sparse tensor in the desired format.

    Raises:
        ValueError: If an unsupported format is specified.
    """
    if from_format == to_format:
        return tensor

    if from_format == "COO":
        if to_format == "CSR":
            return tensor.to_sparse_csr()
        elif to_format == "CSC":
            return tensor.to_sparse_csc()
    elif from_format == "CSR":
        if to_format == "COO":
            return tensor.to_sparse()
        elif to_format == "CSC":
            return tensor.to_sparse().to_sparse_csc()
    elif from_format == "CSC":
        if to_format == "COO":
            return tensor.to_sparse()
        elif to_format == "CSR":
            return tensor.to_sparse().to_sparse_csr()

    raise ValueError(f"Unsupported conversion from {from_format} to {to_format}")

# Function to load OGB dataset and convert sparse formats
def load_and_convert_ogb(dataset_name="ogbn-arxiv", to_format="CSR"):
    dataset = NodePropPredDataset(name=dataset_name)
    split_idx = dataset.get_idx_split()
    graph, labels = dataset[0]

    # Prepare COO tensor
    edge_index = torch.tensor(graph["edge_index"], dtype=torch.int64)
    values = torch.ones(edge_index.shape[1])
    size = (graph["num_nodes"], graph["num_nodes"])
    coo_tensor = torch.sparse_coo_tensor(edge_index, values, size).coalesce()

    # Convert to the desired format
    converted_tensor = convert_sparse_format(coo_tensor, "COO", to_format)

    # Return split indices and converted tensor
    return split_idx, converted_tensor

# Test function
def test_sparse_conversion_with_ogb():
    dataset_name = "ogbn-arxiv"

    # Load and convert to CSR
    split_idx, csr_tensor = load_and_convert_ogb(dataset_name, "CSR")
    print("Converted to CSR format:")
    print("Indptr:", csr_tensor.crow_indices()[:10])
    print("Cols:", csr_tensor.col_indices()[:10])
    print("Values:", csr_tensor.values()[:10])

    # Convert back to COO
    coo_from_csr = convert_sparse_format(csr_tensor, "CSR", "COO").coalesce()
    print("Converted back to COO from CSR:")
    print("Indices:", coo_from_csr.indices()[:, :10])
    print("Values:", coo_from_csr.values()[:10])

    # Load and convert to CSC
    split_idx, csc_tensor = load_and_convert_ogb(dataset_name, "CSC")
    print("Converted to CSC format:")
    print("Indptr:", csc_tensor.ccol_indices()[:10])
    print("Rows:", csc_tensor.row_indices()[:10])
    print("Values:", csc_tensor.values()[:10])

    # Convert back to COO
    coo_from_csc = convert_sparse_format(csc_tensor, "CSC", "COO").coalesce()
    print("Converted back to COO from CSC:")
    print("Indices:", coo_from_csc.indices()[:, :10])
    print("Values:", coo_from_csc.values()[:10])

if __name__ == "__main__":
    test_sparse_conversion_with_ogb()


Converted to CSR format:
Indptr: tensor([ 0,  2,  3, 12, 12, 13, 13, 13, 14, 15])
Cols: tensor([ 52893,  93487, 141692,  18735,  22689,  50646,  80266, 119218, 134600,
        158293])
Values: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Converted back to COO from CSR:
Indices: tensor([[     0,      0,      1,      2,      2,      2,      2,      2,      2,
              2],
        [ 52893,  93487, 141692,  18735,  22689,  50646,  80266, 119218, 134600,
         158293]])
Values: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Converted to CSC format:
Indptr: tensor([  0, 289, 290, 295, 297, 302, 303, 311, 312, 312])
Rows: tensor([ 411,  640, 1162, 1897, 3396, 3787, 4692, 5037, 5537, 5611])
Values: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Converted back to COO from CSC:
Indices: tensor([[     0,      0,      1,      2,      2,      2,      2,      2,      2,
              2],
        [ 52893,  93487, 141692,  18735,  22689,  50646,  80266, 119218, 134600,
         158293]